# Features Selection: Scikit Learn

Collection of features selection techniques provided by Scikit Learn:
- non-supervised tecniques.
- supervised techniques.

In [1]:
## including in the system path
import sys
sys.path.append('../../')  
from tools.reader import csv2df as reader

### arguments

In [2]:
path_input = '../../../datasets/dataset.solar.csv'

### load data

In [3]:
# read
ddt = {'lcol':['dt'], 'sformat': '%Y-%m-%d %H:%M:%S'}
data, dcol = reader(path_input, lindex = ['dt'], ltarget = ['ghi'], ddt = ddt)
# filter (only hforecast = 0 - 23h)
data = data[data.hforecast<24]

## RFECV (supervised)

Feature ranking with recursive feature elimination and cross-validated selection of the best number of features.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.cross_validation import StratifiedKFold
from sklearn.feature_selection import RFECV
clf = SVR(kernel="linear")
features = data[dcol['lx']].values
target = data[dcol['ly'][0]].values
rfecv = RFECV(estimator=clf, step=1, cv=5,
      scoring='r2')
rfecv.fit(features, target)
print("Optimal number of features : %d" % rfecv.n_features_)
print (rfecv.support_)
features=features[:,rfecv.support_]
# Plot number of features VS. cross-validation scores
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.show()

/home/juan/miniconda3/envs/mltools/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
